# Data Agnostic RoBERTa-based Natural Languageto SQL Query Generation



### IMPORTANT: Set the runtime accelerator to use a GPU otherwise the code will run into errors

The code cell below will mount your Google Drive to the colab notebook so that you can use files directly from Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


The below cell creates a copy of the GitHub repository, It lets you use the functions from the repo without having to install it locally

In [2]:
!rm -rf RoBERTa-NL2SQL

GIT_PATH = "https://github.com/DebadityaPal/RoBERTa-NL2SQL"
!git clone "{GIT_PATH}"
%cd RoBERTa-NL2SQL

Cloning into 'RoBERTa-NL2SQL'...
remote: Enumerating objects: 32, done.
remote: Counting objects: 100% (32/32), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 32 (delta 9), reused 7 (delta 3), pack-reused 0
Unpacking objects: 100% (32/32), done.
/content/RoBERTa-NL2SQL


Downloading the dependencies

In [3]:
!pip install records
!pip install transformers==3.4.0

     |████████████████████████████████| 163kB 25.1MB/s 
     |████████████████████████████████| 51kB 8.0MB/s 
  Created wheel for openpyxl: filename=openpyxl-2.4.11-py2.py3-none-any.whl size=222821 sha256=b7404db2eaab888e6344ce9f60ae374280461f31f3742cc3200206f5b3afc39d
  Stored in directory: /root/.cache/pip/wheels/59/44/27/63b211425501ad51d197ff8ed00e9e469e38b9e516cb69b1c2
Successfully built openpyxl
  Found existing installation: openpyxl 2.5.9
    Uninstalling openpyxl-2.5.9:
      Successfully uninstalled openpyxl-2.5.9
     |████████████████████████████████| 1.3MB 17.4MB/s 
     |████████████████████████████████| 2.9MB 46.2MB/s 
     |████████████████████████████████| 890kB 46.5MB/s 
     |████████████████████████████████| 1.2MB 49.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=a6d5107b4d9cc0a7ca011a5cee989c44ad97c6f738306d2ef64da0ef02c5bd2c
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb

In [4]:
import sys
sys.path.insert(0, '/content/drive/My Drive/RoBerta/RoBERTaNL2SQL')

Let us import the libraries that we will be using

In [4]:
import load_data
import torch
import json,argparse
import load_model
import roberta_training
import corenlp_local
import seq2sql_model_testing
import seq2sql_model_training_functions
import model_save_and_infer
import dev_function
import infer_functions
import time
import os
import nltk

from dbengine_sqlnet import DBEngine
from torchsummary import summary
from tqdm.notebook import tqdm
nltk.download('punkt')
from nltk.tokenize import word_tokenize, sent_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


The following cell will set the PyTorch device to a GPU which enables us to use it during runtime.

In [5]:
device = torch.device("cuda")
device

device(type='cuda')

## Loading Data From Files

In [6]:
path_wikisql = "/content/drive/My Drive/RoBerta"
BATCH_SIZE = 8

train_data, train_table, dev_data, dev_table, train_loader, dev_loader = load_data.get_data(path_wikisql, batch_size = BATCH_SIZE)
test_data,test_table,test_loader = load_data.get_test_data(path_wikisql, batch_size = BATCH_SIZE)
zero_data,zero_table,zero_loader = load_data.get_zero_data(path_wikisql, batch_size = BATCH_SIZE)    # Data to test Zero Shot Learning

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


## Loading Models

In [7]:
roberta_model, tokenizer, configuration = load_model.get_roberta_model()          # Loads the RoBERTa Model
seq2sql_model = load_model.get_seq2sql_model(configuration.hidden_size)           # Loads the LSTM based submodels

## Loading the Pre trained weights, skip the below cell if you want to train the model from scratch

In [8]:
pathForUpdatedModels = os.path.join(path_wikisql,"1")
# pathForUpdatedModels = os.path.join(path_wikisql)
print(pathForUpdatedModels)

/content/drive/My Drive/RoBerta/1


In [9]:
path_roberta_pretrained = pathForUpdatedModels + "/model_roberta_best.pt"
path_model_pretrained = pathForUpdatedModels + "/model_best.pt"

if torch.cuda.is_available():
    res = torch.load(path_roberta_pretrained)
else:
    res = torch.load(path_roberta_pretrained, map_location='cpu')

roberta_model.load_state_dict(res['model_roberta'])

if torch.cuda.is_available():
    res = torch.load(path_model_pretrained)
else:
    res = torch.load(path_model_pretrained, map_location='cpu')

seq2sql_model.load_state_dict(res['model'])

<All keys matched successfully>

## Loading the Model Optimizers

##### RoBERTa: Adam Optimizer with learning rate = 0.00001
##### SubModels: Adam Optimizer with learning rate = 0.001

In [10]:
model_optimizer, roberta_optimizer = load_model.get_optimizers(seq2sql_model , roberta_model)

## Below we define a function that prints the metrics in a readable format

In [11]:
def print_result(epoch, acc, dname):
    ave_loss, acc_sc, acc_sa, acc_wn, acc_wc, acc_wo, acc_wvi, acc_wv, acc_lx, acc_x = acc

    print(f'{dname} results ------------')
    print(
        f" Epoch: {epoch}, ave loss: {ave_loss}, acc_sc: {acc_sc:.3f}, acc_sa: {acc_sa:.3f}, acc_wn: {acc_wn:.3f}, \
        acc_wc: {acc_wc:.3f}, acc_wo: {acc_wo:.3f}, acc_wvi: {acc_wvi:.3f}, acc_wv: {acc_wv:.3f}, acc_lx: {acc_lx:.3f}, acc_x: {acc_x:.3f}"
    )

## Training the Model

In [12]:
EPOCHS = 30

In [ ]:
acc_lx_t_best = 0.40            # Creats checkpoint so that a worse model does not get saved
epoch_best = 0                   
for epoch in range(2, EPOCHS):
    acc_train = dev_function.train( seq2sql_model, roberta_model, model_optimizer, roberta_optimizer, tokenizer, configuration, path_wikisql, train_loader)
    acc_dev, results_dev, cnt_list = dev_function.test(seq2sql_model, roberta_model, model_optimizer, tokenizer, configuration, path_wikisql, dev_loader, mode="dev")
    print_result(epoch, acc_train, 'train')
    print_result(epoch, acc_dev, 'dev')
    acc_lx_t = acc_dev[-2]
    if acc_lx_t > acc_lx_t_best:                  # IMPORTANT : Comment out this whole if block if you are using a shortcut to the original
        acc_lx_t_best = acc_lx_t                  #             Drive Folder, otherwise an error will stop the execution of the code.
        epoch_best = epoch                        #             You cannot edit the files in the original folder
                                                  #             Download and Upload a separate copy to change the files.
          
        # save best model
        state = {'model': seq2sql_model.state_dict()}
        if os.path.isdir(os.path.join(path_wikisql, str(epoch))):
          torch.save(state, os.path.join(path_wikisql, str(epoch) , 'model_best.pt'))
        else:
          os.mkdir(os.path.join(path_wikisql, str(epoch)))
          torch.save(state, os.path.join(path_wikisql, str(epoch) , 'model_best.pt'))

        state = {'model_roberta': roberta_model.state_dict()}
        if os.path.isdir(os.path.join(path_wikisql, str(epoch))):
          torch.save(state, os.path.join(path_wikisql, str(epoch) , 'model_roberta_best.pt'))
        else:
          os.mkdir(os.path.join(path_wikisql, str(epoch)))
          torch.save(state, os.path.join(path_wikisql, str(epoch) , 'model_roberta_best.pt'))
    print(f" Best Dev lx acc: {acc_lx_t_best} at epoch: {epoch_best}")

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils.py:264: FutureWarning: `is_pretokenized` is deprecated and will be removed in a future version, use `is_split_into_words` instead.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1709: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


/content/RoBERTa-NL2SQL/seq2sql_model_classes.py:134: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  idxs = topk_multi_dim(torch.tensor(prob_sca),



train results ------------
 Epoch: 2, ave loss: 0.22997043543796755, acc_sc: 0.960, acc_sa: 0.905, acc_wn: 0.980,         acc_wc: 0.909, acc_wo: 0.967, acc_wvi: 0.940, acc_wv: 0.941, acc_lx: 0.763, acc_x: 0.819
dev results ------------
 Epoch: 2, ave loss: None, acc_sc: 0.970, acc_sa: 0.907, acc_wn: 0.981,         acc_wc: 0.916, acc_wo: 0.967, acc_wvi: 0.000, acc_wv: 0.935, acc_lx: 0.783, acc_x: 0.839
 Best Dev lx acc: 0.7832798954993468 at epoch: 2



train results ------------
 Epoch: 3, ave loss: 0.2009621586110871, acc_sc: 0.961, acc_sa: 0.905, acc_wn: 0.982,         acc_wc: 0.913, acc_wo: 0.970, acc_wvi: 0.948, acc_wv: 0.949, acc_lx: 0.771, acc_x: 0.825
dev results ------------
 Epoch: 3, ave loss: None, acc_sc: 0.972, acc_sa: 0.911, acc_wn: 0.985,         acc_wc: 0.929, acc_wo: 0.972, acc_wvi: 0.000, acc_wv: 0.950, acc_lx: 0.801, acc_x: 0.854
 Best Dev lx acc: 0.8006175038593991 at epoch: 3



train results ------------
 Epoch: 4, ave loss: 0.18069479206446223, acc_sc: 0.966, acc_sa: 0.906, acc_wn: 0.984,         acc_wc: 0.920, acc_wo: 0.973, acc_wvi: 0.956, acc_wv: 0.956, acc_lx: 0.786, acc_x: 0.839
dev results ------------
 Epoch: 4, ave loss: None, acc_sc: 0.972, acc_sa: 0.907, acc_wn: 0.986,         acc_wc: 0.931, acc_wo: 0.975, acc_wvi: 0.000, acc_wv: 0.954, acc_lx: 0.800, acc_x: 0.856
 Best Dev lx acc: 0.8006175038593991 at epoch: 3


## Testing The Model

In [ ]:
acc_dev, results_dev, _ = dev_function.test(seq2sql_model, roberta_model, model_optimizer, tokenizer, configuration, path_wikisql, dev_loader, mode="dev")
acc_test, results_test, _ = dev_function.test(seq2sql_model, roberta_model, model_optimizer, tokenizer, configuration, path_wikisql, test_loader, mode="test")
acc_zero, results_zero, _ = dev_function.test(seq2sql_model, roberta_model, model_optimizer, tokenizer, configuration, path_wikisql, zero_loader, mode="test")

print_result('test', acc_dev, 'dev')
print_result('test', acc_test, 'test')
print_result('test', acc_zero, 'zero')

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils.py:264: FutureWarning: `is_pretokenized` is deprecated and will be removed in a future version, use `is_split_into_words` instead.
  FutureWarning,
/content/RoBERTa-NL2SQL/seq2sql_model_classes.py:134: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  idxs = topk_multi_dim(torch.tensor(prob_sca),
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1639: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")



dev results ------------
 Epoch: test, ave loss: None, acc_sc: 0.972, acc_sa: 0.906, acc_wn: 0.989,         acc_wc: 0.941, acc_wo: 0.979, acc_wvi: 0.000, acc_wv: 0.959, acc_lx: 0.813, acc_x: 0.869
test results ------------
 Epoch: test, ave loss: None, acc_sc: 0.968, acc_sa: 0.905, acc_wn: 0.982,         acc_wc: 0.933, acc_wo: 0.972, acc_wvi: 0.000, acc_wv: 0.954, acc_lx: 0.805, acc_x: 0.860
zero results ------------
 Epoch: test, ave loss: None, acc_sc: 0.967, acc_sa: 0.896, acc_wn: 0.981,         acc_wc: 0.928, acc_wo: 0.967, acc_wvi: 0.000, acc_wv: 0.945, acc_lx: 0.786, acc_x: 0.845


## Test You Own Queries!

In [ ]:
nlu = "Which year did the band release the Song 'Wake me Up'?"

# Specify the Table Schema
table_id = '1-10015132-16'
headers = ['Band', 'Song', 'Studio', 'Year', 'Awards']
types = ['text', 'text', 'text', 'text', 'text']

In [ ]:
pr_sql_i =  infer_functions.infer(
                nlu,
                table_id, headers, types, tokenizer, 
                seq2sql_model, roberta_model, configuration, max_seq_length=222,
                num_target_layers=2,
                beam_size=4
            )

START ============================================================= 
[['Band', 'Song', 'Studio', 'Year', 'Awards']]
nlu: ["Which year did the band release the Song 'Wake me Up'?"]
pr_sql_i : [{'agg': 0, 'sel': 3, 'conds': [[1, 0, "'wake me up"]]}]
pr_sql_q : [["SELECT (Year) FROM 1-10015132-16 WHERE Song = 'wake me up"]]
---------------------------------------------------------------------


/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils.py:264: FutureWarning: `is_pretokenized` is deprecated and will be removed in a future version, use `is_split_into_words` instead.
  FutureWarning,
/content/RoBERTa-NL2SQL/seq2sql_model_classes.py:134: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  idxs = topk_multi_dim(torch.tensor(prob_sca),
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1639: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Use the cell below to delete the cloned repository, this will free up the space used.

In [ ]:
!rm -rf RoBERTa-NL2SQL

In [ ]:
os.path.isdir('/content/drive/My Drive/RoBerta/0') 

True

In [ ]:
import os

In [ ]:
os.mkdir('/content/drive/My Drive/RoBerta/0')